In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from collections import Counter, defaultdict
from sklearn.utils import check_random_state

#### Stratified k-fold splitting function

In [ ]:
#Slightly modified splitting function but not my code. Can be found here https://www.kaggle.com/underwearfitting/how-to-split-folds-given-new-drug-id
def split(X, y, groups, numSplits = 5, random_state=None):

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std(
                [y_counts_per_fold[i][label] / y_distr[label] for i in range(numSplits)]
            )
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)

    rnd = check_random_state(random_state)

    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

        y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
        groups_per_fold = defaultdict(set)

        groups_and_y_counts = list(y_counts_per_group.items())
        rnd.shuffle(groups_and_y_counts)

        for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
            best_fold = None
            min_eval = None
            for i in range(numSplits):
                fold_eval = eval_y_counts_per_fold(y_counts, i)
                if min_eval is None or fold_eval < min_eval:
                    min_eval = fold_eval
                    best_fold = i
            y_counts_per_fold[best_fold] += y_counts
            groups_per_fold[best_fold].add(g)

        all_groups = set(groups)
        for i in range(numSplits):
            train_groups = all_groups - groups_per_fold[i]
            test_groups = groups_per_fold[i]

            train_indices = [i for i, g in enumerate(groups) if g in train_groups]
            test_indices = [i for i, g in enumerate(groups) if g in test_groups]

            yield train_indices, test_indices

In [ ]:
base_dir = "./ranzcr_clip_catheter_line_classification/"
train_df = pd.read_csv(os.path.join(base_dir, "train.csv"))

#concat all the labels into a single number
target_cols = train_df.drop(['StudyInstanceUID', 'PatientID'],axis=1).columns.values.tolist()
targets = train_df[target_cols].astype(str)
# create a new col to store the label
train_df['combined_label'] = ''
for i in range(targets.shape[1]):
    train_df['combined_label'] += targets.iloc[:,i]
# how many of each label there are
print(train_df.combined_label.value_counts())
#encode training labels
train_df['combined_label'] = LabelEncoder().fit_transform(train_df['combined_label'])


train_df['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(split(X=train_df, y=train_df.combined_label, groups=train_df.PatientID, random_state=0)):
    train_df.loc[valid_idx, 'fold'] = int(i)

#save training folds csv
train_df.to_csv('train_folds.csv', index=True)